# Running Dask on the cluster

The dask frameworks enabling users to parallelize internal systems
Not all computations fit into a big dataframe. Dask exposes lower-level APIs letting you build custom systems for in-house applications. This helps parallelize python processes and dramatically accelerate their performance

Dask Kubernetes deploys Dask workers on Kubernetes clusters using native Kubernetes APIs. It is designed to dynamically launch short-lived deployments of workers during the lifetime of a Python process.

check out this link https://kubernetes.dask.org/en/latest/

In [1]:
!pip install dask-kubernetes

When user runs dask the frameworks start one or more pods running in parallel on the cluster. Users can define the number of nodes and the minimun and maximum number of pods that the dask framework opens up
Scale to zero is achieved by setting the minimum = 0. Setting it to zero delete the pods once the job is done and free up the resources 

In [2]:
from dask_kubernetes import KubeCluster

cluster = KubeCluster.from_yaml('worker-spec.yml')
cluster.scale_up(4)  # specify number of nodes explicitly

cluster.adapt(minimum=2, maximum=5)  # or dynamically scale based on current workload

/conda/lib/python3.6/site-packages/bokeh/themes/theme.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  json = yaml.load(f)
distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
/User/.pythonlibs/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)
distributed.scheduler - INFO -   Scheduler at:  tcp://10.233.81.31:43201
distributed.scheduler - INFO -   dashboard at:                    :42621


distributed.scheduler - INFO - Retire worker names (0, 1)
distributed.deploy.adaptive - INFO - Retiring workers [0, 1]


to view the pods that are running

In [3]:
!kubectl  -n default-tenant get pods | grep dask

dask-iguazio-4e54e604-bcfqrc                       1/1     Running     0          10s
dask-iguazio-4e54e604-bldmph                       1/1     Running     0          10s


In [4]:
# Example usage
import distributed
import dask.array as da

# Connect dask to the cluster
client = distributed.Client(cluster)

# Create an array and calculate the mean
array = da.ones((1000, 1000, 1000), chunks=(100, 100, 10))
print(array.mean().compute())  # Should print 1.0

distributed.scheduler - INFO - Receive client connection: Client-ec580b48-0b0c-11ea-9bc2-019f8a8ab19d
distributed.core - INFO - Starting established connection
distributed.core - INFO - Event loop was unresponsive in Scheduler for 3.09s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.scheduler - INFO - Register tcp://10.233.76.67:33187
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.233.76.67:33187
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Retire worker names (2, 3, 4)
distributed.deploy.adaptive - INFO - Retiring workers [2, 3, 4]
distributed.scheduler - INFO - Remove worker tcp://10.233.76.67:33187
distributed.core - INFO - Removing comms to tcp://10.233.76.67:33187
distributed.scheduler - INFO - Lost all workers
distributed.scheduler - INFO - Register tcp://10.233.111.64:33079
distributed.scheduler - INFO - Starting 

1.0


distributed.scheduler - INFO - Register tcp://10.233.76.68:46786
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.233.76.68:46786
distributed.core - INFO - Starting established connection
